# Imports


In [1]:
using BenchmarkTools
using XLSX
using DataFrames
using BilevelJuMP, HiGHS, Ipopt, SCIP
using BenchmarkTools

### Funcion que crea o sobrescribe el xlsx

In [2]:
function serialize_in_xlsx(df,file_name::String)
    sheet_name="Data"
    all_name="$file_name.xlsx"
    if  !isfile(file_name) # Si no existe creo y lo guardo
        XLSX.writetable(all_name, sheet_name =>df; overwrite=true )
        return all_name
    end
        # Si existe el archivo
        # Leer la tabla desde el archivo Excel
        tabla = XLSX.readtable(all_name,sheet_name)
        # Convertir en un DataFrame
        old_df=DataFrame(tabla)
        # Concatenar los Dataframes
        new_df=vcat(old_df, df)
        # Reescribir y guardar el nuevo xlsx
        XLSX.writetable(all_name, sheet_name => new_df; overwrite=true )
        return all_name
end
    


serialize_in_xlsx (generic function with 1 method)

Funcion que despues escribe en un excel el ejemplo

In [3]:
function add_to_dataframe_vars_value(df,var_vector::Vector,var_name::String)
    # Insertar una nueva columna al final
    for i in 1:length(var_vector)
        insertcols!(df, ncol(df) + 1,"$var_name: $i"=>BilevelJuMP.value(var_vector[i]) )
    end
    return df
end

add_to_dataframe_vars_value (generic function with 1 method)

In [4]:
function create_dataframe(model,x,y,name_optimize_method::String)
    println("optimize Method: $name_optimize_method")
    # Escribir el resultado de la solucion primal
    p_s=string(BilevelJuMP.primal_status(model))
    println("Primal Status: $p_s")
    # Escribir el codigo y tipo de terminacion
    t_s=string(BilevelJuMP.termination_status(model))
    println("Termination Status: $t_s")
    # Objetive value
    o_v=BilevelJuMP.objective_value(model)
    println("Objetive Value $o_v")
    # Lider Vars Value
    l_vars=BilevelJuMP.value.(x)
    println("Lider Vars: $l_vars")
    # Follower Vars Value
    f_vars=BilevelJuMP.value.(y)
    println("Followers Vars Value $f_vars")

    resultado=@benchmark optimize!(model)
    # Extraer los tiempos y otros datos relevantes
    tiempos = resultado.times  # Tiempos en nanosegundos
    min_tiempo = minimum(tiempos) / 1e9  # Tiempo mínimo en segundos
    max_tiempo = maximum(tiempos) / 1e9  # Tiempo máximo en segundos
    promedio_tiempo = mean(tiempos) / 1e9  # Tiempo promedio en segundos
    
    # Extraer el uso de recursos
    num_asignaciones = sum(resultado.allocs)  # Total de asignaciones de memoria
    memoria_usada = maximum(resultado.memory) / (1024^2)  # Memoria máxima usada en MB

    # Crear un DataFrame para almacenar los resultados
    df_resultados = DataFrame(
    Optimize_Method=name_optimize_method,
    Estatus_Primal=p_s,
    Estatus_Terminación=t_s,
    Valor_Objetivo=o_v,
    
    Mínimo=min_tiempo,
    Máximo=max_tiempo,
    Promedio=promedio_tiempo,
    Asignaciones=num_asignaciones,
    Memoria_MB=memoria_usada
    )
    # Añadir el valor de las x_s 
    df_resultados=add_to_dataframe_vars_value(df_resultados,x,"X_s")
    # Anadir el valor de las y_s
    df_resultados=add_to_dataframe_vars_value(df_resultados,y,"Y_s")

    return df_resultados

end

create_dataframe (generic function with 1 method)

In [5]:
function make_experiment(model_with_optimizer,x_s,y_s,optimizer_name::String,problem_name::String)
    # Mandar a optimizar el modelo 
    optimize!(model)
    # Crear el dataframe de respuesta
    df= create_dataframe(model,x_s,y_s,optimizer_name)
    # Ahora serializar en un xlxs los datos con el nombre del modelo
    # Guardar el DataFrame en un archivo Excel
    file_name=serialize_in_xlsx(df,problem_name)

 

    # Mostrar mensaje de confirmación
    println("El DataFrame ha sido guardado en $file_name en la hoja $problem_name.")
end

make_experiment (generic function with 1 method)

Sacar el tiempo y uso de memoria

# Importar los modulos necesarios (BilevelJuMP y los optimizadores)

# Crear el modelo del experimento

In [6]:
model = BilevelModel()

An Abstract JuMP Model
Feasibility problem with:
Variables: 0
Upper Constraints: 0
Lower Constraints: 0
Bilevel Model
Solution method: BilevelJuMP.NoMode{Float64}
No solver attached

In [7]:
BilevelJuMP.@variable(Lower(model), x)
BilevelJuMP.@variable(Upper(model), y)

y

In [8]:
BilevelJuMP.@objective(Upper(model), Min, 3x + y)
BilevelJuMP.@constraints(Upper(model), begin
    x <= 5
    y <= 8
    y >= 0
end)

BilevelJuMP.@objective(Lower(model), Min, -x)
BilevelJuMP.@constraints(Lower(model), begin
    c1,  x +  y <= 8
    c2, 4x +  y >= 8
    c3, 2x +  y <= 13
    c4, 2x - 7y <= 0
end)

(c1 : x + y ≤ 8, c2 : 4 x + y ≥ 8, c3 : 2 x + y ≤ 13, c4 : 2 x - 7 y ≤ 0)

## Using HiGHS Optimizer

In [9]:
BilevelJuMP.set_optimizer(model, HiGHS.Optimizer)

BilevelJuMP.set_mode(model,
    BilevelJuMP.BigMMode(primal_big_M = 100, dual_big_M = 100))

#BilevelJuMP.optimize!(model)
#
#BilevelJuMP.objective_value(model)
#@assert abs(BilevelJuMP.objective_value(model) - (3 * (7/2 * 8/15) + 8/15)) < 1e-1 # src

An Abstract JuMP Model
Minimization problem with:
Variables: 2
Upper objective function type: 
GenericAffExpr{Float64, BilevelVariableRef}
Lower objective function type: 
GenericAffExpr{Float64, BilevelVariableRef}
Upper Constraints: 3
Lower Constraints: 4
Bilevel Model
Solution method: BilevelJuMP.BigMMode{Float64}(false, 100.0, 100.0, BilevelJuMP.ComplementBoundCache(Dict{MathOptInterface.VariableIndex, BilevelJuMP.BilevelVariableInfo}(), Dict{MathOptInterface.VariableIndex, BilevelJuMP.BilevelVariableInfo}(), Dict{MathOptInterface.ConstraintIndex, BilevelJuMP.BilevelConstraintInfo}(), Dict{MathOptInterface.VariableIndex, BilevelJuMP.BilevelVariableInfo}()))
Solver name: HiGHS
Names registered in the model: c1, c2, c3, c4, x, y

In [10]:
make_experiment(model,[x],[y],"Highs- BigM","Test")

Running HiGHS 1.8.1 (git hash: 4a7f24ac6): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 9e+02]
  Cost   [1e+00, 3e+00]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 1e+02]
Assessing feasibility of MIP using primal feasibility and integrality tolerance of       1e-06
Solution has               num          max          sum
Col     infeasibilities      0            0            0
Integer infeasibilities      0            0            0
Row     infeasibilities      4           13           30
Row     residuals            0            0            0
Attempting to find feasible solution by solving LP for user-supplied values of discrete variables
Coefficient ranges:
  Matrix [1e+00, 9e+02]
  Cost   [1e+00, 3e+00]
  Bound  [0e+00, 0e+00]
  RHS    [1e+00, 1e+02]
Presolving model
13 rows, 6 cols, 24 nonzeros  0s
Problem status detected on presolve: Infeasible
Model status        : Infeasible
Objective value     :  0.0000000000e+00
HiGHS run time      :      